In [ ]:
import preprocess
import sys
from Gabor import *
import os
import numpy as np
import cv2
from skimage import feature as skif
from sklearn import svm

In [ ]:
class LBP(object):
    # 使用LBP+SVM实现表情识别

    def load_image(self, foler):
        """
        根据给定的目录，读取当前目录下的所有图片
        :param foler:
        :return:
        """
        categories = os.listdir(foler)  # 得到当前foler文件夹下所有的目录
        imags = []
        labels = []
        for category in categories:
            now_folder = os.path.join(foler, category)
            subsdirectories = os.listdir(now_folder)
            for subsdirectory in subsdirectories:
                now_path = os.path.join(now_folder, subsdirectory)
                image = cv2.imread(now_path)
                grab = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                imags.append(self.get_lbp(grab))
                labels.append(category)
        images_array = np.array(imags)
        return images_array, labels

    def get_lbp(self, image):
        """
        获取给定图片的LBP，划分成几个区域后
        :param rgb:
        :return:
        """
        gridx = 6
        gridy = 6
        widx = 8
        widy = 8
        hists = []
        for i in range(gridx):
            for j in range(gridy):
                mat = image[i * widx: (i + 1) * widx, j * widy: (j + 1) * widy]
                lbp = skif.local_binary_pattern(mat, 8, 1, 'uniform')
                max_bins = 10
                hist, _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))
                hists.append(hist)
        out = np.array(hists).reshape(-1, 1)
        return out

    def label2number(self, label_list):
        label = np.zeros(len(label_list))
        label_unique = np.unique(label_list)
        num = label_unique.shape[0]
        for k in range(num):
            index = [i for i, v in enumerate(label_list) if v == label_unique[k]]
            label[index] = k
        return label, label_unique

    def train_test(self, train_data, train_label, test_data, test_label):
        train_data = np.squeeze(train_data, axis=-1)
        test_data = np.squeeze(test_data, axis=-1)
        svr_rbf = svm.SVC(kernel='rbf', C=1.0, gamma='auto')
        svr_rbf.fit(train_data, train_label)
        pred = svr_rbf.predict(test_data)
        print(np.sum(pred == test_label))
